In [1]:
import json
import numpy as np
import fastai
import fastai.tabular
from fastai.vision.all import *
import matplotlib.pyplot as plt
from mocatml.utils import *
from mocatml.data import *
from tsai.basics import *
from tsai.inference import load_learner
from CNN import *

In [7]:
with open("data_config.json", "r") as outfile:
    data = json.load(outfile, strict=False)
extent = list(data['extent'])
data_numpy = np.load('data.npy')

d, n_epochs, num_sim, n_epochs = 256, 35, 100, 100
model_name = 'TSTPlus'
learn = load_learner(f'time_model/{model_name}_d_{d}_epoch_{n_epochs}.pkl')
model = AutoEncoder(d)
load_model(f'model/d_{d}_epoch_35.pkl', model, None)

In [8]:
with open(f'model/indices_d_{d}_epoch_35.txt', "r") as f:
    indices = f.read().split()
    
split = float(indices[0])
indices = [int(i) for i in indices[1:]]

train_indices = indices[:int(split * num_sim)]
val_indices = indices[int(split * num_sim):]

data_enc = []
for i in range(num_sim):
    encoded = model.encrypt(torch.tensor(data_numpy[i], dtype=torch.float32).unsqueeze(0).transpose(0, 1))
    data_enc.append(encoded.detach().numpy())
data_enc = np.array(data_enc)
sh = data_enc.shape

data_enc_reshaped = data_enc.reshape((sh[0], sh[-1], sh[1]))
X_train, y_train = apply_sliding_window_3d(data_enc_reshaped[train_indices], 
                                           window_len=10,
                                           stride=1,
                                           seq_first=False,
                                           horizon=1)
X_valid, y_valid = apply_sliding_window_3d(data_enc_reshaped[val_indices], 
                                           window_len=10,
                                           stride=1,
                                           seq_first=False,
                                           horizon=1)

X, y, splits = combine_split_data([X_train, X_valid], [y_train, y_valid])
X.shape, y.shape, splits

((242600, 256, 10),
 (242600, 256, 1),
 ((#230470) [0,1,2,3,4,5,6,7,8,9...],
  (#12130) [230470,230471,230472,230473,230474,230475,230476,230477,230478,230479...]))

In [ ]:
cb_handler = CallbackHandler(newlearner.callbacks + [], None)
losses, acc = fastai.basic_train.validate(
    newlearner.model, 
    newlearner.data.test_dl, 
    newlearner.loss_func, 
    cb_handler,  # This is necessary
    average=False)